### Sources

* English sentences training file: https://github.com/sigmorphon/2022SegmentationST/blob/ac161e1107e423577e922b05f8c43c6ebad6722a/data/eng.sentence.train.tsv
* English sentences test file: https://github.com/sigmorphon/2022SegmentationST/blob/ac161e1107e423577e922b05f8c43c6ebad6722a/data/eng.sentence.test.tsv
* English sentences test gold file: https://github.com/sigmorphon/2022SegmentationST/blob/ac161e1107e423577e922b05f8c43c6ebad6722a/data/eng.sentence.test.gold.tsv
* Tsez sentences training data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Tsez/ddo-train-track2-uncovered
* Tsez sentences test data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Tsez/ddo-test-track2-uncovered
* Natagu sentences training data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Natugu/ntu-train-track2-uncovered
* Natagu sentences test data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Natugu/ntu-test-track2-uncovered
* Lezgi sentences training data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Lezgi/lez-train-track2-uncovered
* Lezgi sentences test data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Lezgi/lez-test-track2-uncovered
* Gitskan sentences training data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Gitksan/git-train-track2-uncovered
* Gitskan sentences test data: https://github.com/sigmorphon/2023glossingST/blob/259be10acad334feb36869eec05263b8b8fab436/data/Gitksan/git-test-track2-uncovered
* Evaluation script: https://github.com/sigmorphon/2022SegmentationST/blob/ac161e1107e423577e922b05f8c43c6ebad6722a/evaluation/evaluate.py


# Morfessor Setup

In [ ]:
%pip install pandas morfessor 

In [3]:
#unsupervised
import pandas as pd
import morfessor
import subprocess
import os

def train(type="standard", morph_length=-1, num_morph_types=-1, bin_model_path="models\m_morfessor.bin", training_file_path="data\\training_morfessor_cli.txt"):
  if type=="standard":
    train_res = subprocess.run(['morfessor-train.bat', '-s', bin_model_path, '--traindata-list', training_file_path])
  elif type=="morph_length" and morph_length>0:
    train_res = subprocess.run(['morfessor-train.bat', '-s', bin_model_path,'--morph-length', f'{morph_length}', '--traindata-list', training_file_path])
  elif type=="num_morph_types" and num_morph_types>0:
     train_res = subprocess.run(['morfessor-train.bat', '-s', bin_model_path,'--num-morph-types', f'{num_morph_types}', '--traindata-list', training_file_path])  
  else:
     print("Error in the training arguments!")


def segment(bin_model_path="models\m_morfessor.bin", input_tsv="data\\eng.sentence.test.gold.tsv", output_tsv="outputs\\eng.sentence.test.morfessor_guess.tsv"):

  io = morfessor.MorfessorIO()
  model = io.read_binary_model_file(bin_model_path)

  df_guess = pd.read_csv(input_tsv, sep='\t', header=None)
  data_guess = df_guess[0].astype(str)
  i = 0

  #segment on words

  for entry in data_guess:
    sent = ''
    j = 0
    words = entry.split() #list of words
    length_sent = len(words)  #number of words

    for word in words:
      list_word = model.viterbi_segment(word)[0]
      k = 0
      length_word = len(list_word)  #number of morphs

      for morph in list_word:
        if k != length_word - 1:  #not last morph
          sent += (morph + ' @@')
          k+=1
        else: #last morph
          if j != length_sent - 1:
            sent += (morph + ' ')
          else :
            sent +=morph
      j+=1

    df_guess[1][i] = sent
    i+=1

  df_guess.to_csv(output_tsv, sep='\t', header=None, index = False)


def evaluate(type="standard", gld="data\\eng.sentence.test.gold.tsv", gs="outputs\\eng.sentence.test.morfessor_guess.tsv", store="outputs\\eng.output.json", morph_length=-1, num_morph_types=-1, training=False):
  import argparse
  args = argparse.Namespace(
      gold=gld,
      guess=gs,
      output=store,
      category=False 
  )
  import evaluate, json, os
  stats = evaluate.main(args)
  model_name = "morfessor_"
  if training: 
     model_name += "training_"
  if type=="standard":
     model_name += "standard"
  elif type=="morph_length" and morph_length>0:
     model_name += f"len_{morph_length}"
  elif type=="num_morph_types" and num_morph_types>0:
     model_name += f"types_{num_morph_types}"
  else:
     print("Error in arguments!")
  new_stats = {"model": model_name}
  new_stats.update(stats)
  data = {"data": []}
  if os.path.exists(args.output):
      with open(args.output, 'r') as output_file:
          data = json.load(output_file)

  # Skip adding data point if already present
  if not any(item.get("model") == model_name for item in data["data"]):
      data["data"].append(new_stats)
      data["data"] = sorted(data["data"], key=lambda x: x["model"])
      with open(args.output, 'w') as output_file:
          json.dump(data, output_file, indent=4)


<>:7: SyntaxWarning: invalid escape sequence '\m'
<>:18: SyntaxWarning: invalid escape sequence '\m'
<>:7: SyntaxWarning: invalid escape sequence '\m'
<>:18: SyntaxWarning: invalid escape sequence '\m'
C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:7: SyntaxWarning: invalid escape sequence '\m'
  def train(type="standard", morph_length=-1, num_morph_types=-1, bin_model_path="models\m_morfessor.bin", training_file_path="data\\training_morfessor_cli.txt"):
C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:18: SyntaxWarning: invalid escape sequence '\m'
  def segment(bin_model_path="models\m_morfessor.bin", input_tsv="data\\eng.sentence.test.gold.tsv", output_tsv="outputs\\eng.sentence.test.morfessor_guess.tsv"):


# Apply Morfessor to the english sentences test dataset

In [ ]:
#Add the standard evaluation without any flags to the JSON file
train()
segment()
evaluate()

In [ ]:
# Add the evaluations for different --morph-length values to the JSON file

# Find the average length of a word in the training file. Avg. morph length <= Avg. word length
total_len = 0
total_words = 0
with open("data\\training_morfessor_cli.txt", "r", encoding="utf-8") as f:
    for word in f: 
        total_words +=1
        total_len += len(word)
avg_len = 0
if total_words!=0:
    avg_len = int(total_len / total_words)
print(avg_len)

In [ ]:
for i in range(1,avg_len + 1):
    train(type="morph_length", morph_length=i)
    segment()
    evaluate(type="morph_length", morph_length=i)

In [ ]:
#Calculate how many distinct words are in the training file. num_morph_types <= num_distinct_words
distinct_words = []
with open("data\\training_morfessor_cli.txt", "r", encoding="utf-8") as file:
    for word in file:
        if word not in distinct_words:
            distinct_words.append(word)
num_distinct_words = len(distinct_words)
print(num_distinct_words)

In [ ]:
for j in range(500, num_distinct_words, 1000):
    train(type="num_morph_types", num_morph_types=j)
    segment()
    evaluate(type="num_morph_types", num_morph_types=j)

# Apply Morfessor to the english sentences training dataset 

In [ ]:
#Use morfessor to segment the training data itself 
train()
segment(input_tsv="data\\eng.sentence.train.tsv", output_tsv="outputs\\eng.sentence.train.morfessor_guess.tsv")
evaluate(gld="data\\eng.sentence.train.tsv", gs="outputs\\eng.sentence.train.morfessor_guess.tsv", training=True)

In [ ]:
#Record the f-score for the training data for different --morph-length values
for i in range(1,avg_len + 2):
    train(type="morph_length", morph_length=i)
    segment(input_tsv="data\\eng.sentence.train.tsv", output_tsv="outputs\\eng.sentence.train.morfessor_guess.tsv")
    evaluate(type="morph_length", morph_length=i, training=True, gld="data\\eng.sentence.train.tsv", gs="outputs\\eng.sentence.train.morfessor_guess.tsv")

In [ ]:
#Record the f-score for the training data for different --num-morph-types values
for j in range(500, num_distinct_words, 1000):
    train(type="num_morph_types", num_morph_types=j)
    segment(input_tsv="data\\eng.sentence.train.tsv", output_tsv="outputs\\eng.sentence.train.morfessor_guess.tsv")
    evaluate(type="num_morph_types", num_morph_types=j, training=True, gld="data\\eng.sentence.train.tsv", gs="outputs\\eng.sentence.train.morfessor_guess.tsv")

# Using Morfessor on 4 low-resource languages

In [4]:
# Add the standard evaluations for the 4 low-resource languages to the data
langs = ["ddo", "git", "lez", "ntu"]
for lang in langs: 
    train(training_file_path=f"data\\{lang}.train.cli.txt")
    segment(input_tsv=f"data\\{lang}.test.gold.tsv", output_tsv=f"outputs\\{lang}.sentence.test.morfessor_guess.tsv")
    evaluate(gld=f"data\\{lang}.test.gold.tsv", gs=f"outputs\\{lang}.sentence.test.morfessor_guess.tsv", store=f"outputs\\{lang}.output.json")    

C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	16.13
f_measure	35.93
precision	37.95
recall	34.11


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	14.43
f_measure	42.72
precision	45.92
recall	39.94


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	11.89
f_measure	43.12
precision	40.93
recall	45.55


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	10.16
f_measure	51.12
precision	46.83
recall	56.27


In [ ]:
# Add the evaluations for different morph lengths for the 4 low-resource languages to the data
for lang in langs: 
        for i in range(1,7):        
                train(training_file_path=f"data\\{lang}.train.cli.txt", type="morph_length", morph_length=i)
                segment(input_tsv=f"data\\{lang}.test.gold.tsv", output_tsv=f"outputs\\{lang}.sentence.test.morfessor_guess.tsv")
                evaluate(gld=f"data\\{lang}.test.gold.tsv", gs=f"outputs\\{lang}.sentence.test.morfessor_guess.tsv", store=f"outputs\\{lang}.output.json", type="morph_length", morph_length=i)  

C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	49.29
f_measure	16.17
precision	10.82
recall	32.00


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	21.77
f_measure	33.27
precision	28.32
recall	40.33


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent


...



category: all
distance	9.20
f_measure	53.72
precision	51.07
recall	56.65


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	9.07
f_measure	52.84
precision	55.24
recall	50.64


C:\Users\admin\AppData\Local\Temp\ipykernel_11316\1822670189.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent




category: all
distance	9.31
f_measure	49.97
precision	61.98
recall	41.86
